In [1]:
%autosave 0

Autosave disabled


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


In [4]:
import os
import pandas as pd
import numpy as np
import re
import collections
import pickle
from ast import literal_eval
import random
import scipy
import math
import datetime
import subprocess

from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

#Functions

In [5]:
# arrange sequence of tour

def arrange_tour(solver_tour, start_end_points):

    #Get the start and end point of the specific district of the instance
    start_PP, end_PP = start_end_points 
    arranged_tour = 0

    #FUNCTION TO REARRANGE THE LIST WITH CASE OF SAME START AND END POSTPOINT
    def rearrange_list(numbers, specific_value):
        # Find the index of the specific value in the list
        index = numbers.index(specific_value)
        
        # Check if the chosen element is repeated twice consecutively
        if numbers[index] == numbers[index + 1]:
            # Split the list into two parts based on the index
            part1 = numbers[:index]
            part2 = numbers[index + 1:]
            
            # Rearrange the list by concatenating the parts and adding the specific value at the beginning and end
            rearranged_list = part2 + part1 + [specific_value]
        else:
            # Split the list into two parts based on the index
            part1 = numbers[:index]
            part2 = numbers[index:]
            
            # Rearrange the list by concatenating the parts in reverse order
            rearranged_list = part2 + part1
        
        return rearranged_list
    
    
    #FUNCTION TO REARRANGE THE LIST WITH CASE OF DIFFERENT START AND END POSTPOINT
    def rearrange_circular_tour(tour, start_point, end_point):
    # Find the index of the start point in the tour list
        start_index = tour.index(start_point)
            
        # Find the index of the end point in the tour list
        end_index = tour.index(end_point)
                
        # Check if the start and end points are already at the first and last positions
        if start_index == 0 and end_index == len(tour) - 1:
            return tour
                
        # Split the tour list into two parts based on the start and end indices
        if start_index < end_index:
            part1 = tour[0:start_index + 1]
            part1.reverse()

            part2 = tour[start_index + 1:]
            part2.reverse()
                
        else:
            part1 = tour[start_index:] + tour[:end_index+1]
            part2 = tour[end_index+1:start_index]
                
        # Rearrange the tour list by concatenating the parts
        rearranged_tour = part1 + part2
        
        return rearranged_tour
    


    #Case of start and end point equal
    if start_PP == end_PP:
        #If the start and end point are already in correct positions return list as it is                                                                              
        if start_PP == solver_tour[0] and end_PP == solver_tour[len(solver_tour)-1]:                    
            arranged_tour = solver_tour
        else: 
            #Check if start or end point (doesn't matter they have the same value) appears less than two times on the list
            if list.count(solver_tour, start_PP) < 2:
                #If if is not repeated. Check if at least start_point is in first_position.                                                   
                if start_PP == solver_tour[0]: 
                    #If start_point is in first position just append end point at the end of list.                                                         
                    arranged_tour = solver_tour + [end_PP]
                else: 
                    #Rearrange the tour first with start_PP at the beginning of the list before appending end_PP
                    arranged_tour = rearrange_list(solver_tour, start_PP)
                    arranged_tour = arranged_tour + [end_PP]               
            else: 
                #Rearrange the tour in the correct order
                arranged_tour = rearrange_list(solver_tour, start_PP)
    #Case of start and end point different
    else:
       arranged_tour = rearrange_circular_tour(tour = solver_tour, start_point = start_PP, end_point = end_PP)
    
    return arranged_tour

In [6]:
#FUNCTION TO GET SUBSET OF A COMPLETE TOUR AND ITS LENGTH KEEPING SEQUENCE BASED ON INSTANCES VOLUMES

#INPUTS:    Complete Tour (DP or Concord), Complete Tour info (DP or Concord), 
#           Distance Matrix for the instance (generated from instance generation function), Mapping of PostPoints Needed (Dictionary obtained from instance generation function)
#OUTPUTS:   Tour Sequence Fixed, Tour Length, Difference between New Tour and Complete Tour



def new_tour_sequence_fixed(complete_tour, dm_instance, mapping_pp_needed):
    #get the list of only the Postpoint needed for the specific instance
    list_pp_needed = list(mapping_pp_needed.values())

    #Keep from complete_tour only the needed PostPoints according to the list of PostPoints needed mantaining sequence
    tour_seq_fixed = [x for x in complete_tour if x in list_pp_needed]

    #Generate a dataframe from the distance matrix with the correct row-column combination based on PostPoints required
    df_dm_instance = pd.DataFrame(dm_instance)
    df_dm_instance = (df_dm_instance.rename(columns= mapping_pp_needed)).rename(index = mapping_pp_needed)

    #FUNCTION to compute length of new tour (Adapting using logic of function already created for default_route calculation: get_tour_length())
    
    def get_seq_tour_length(sequence_list, distance_df):
        tour_length = 0 
        for i in range(len(sequence_list)-1):
            # Specify the specific values for row and column in the dataframe
            PostPoint_predecesor = sequence_list[i]
            PostPoint_succesor = sequence_list[i+1]

            # Get the distance value needed based on the Post Point combination or raise error
            try:
                distance_value = distance_df[PostPoint_predecesor][PostPoint_succesor]
            except: 
                print('ERROR: No distance for PostPoint combination')

            # Aggreate sum value until end of loop to compute total tour length
            tour_length += distance_value
            
        return round(tour_length,2)
    
    #Calculate tour length of new tour which skips PostPoints
    tour_seq_fixed_length = get_seq_tour_length(tour_seq_fixed, df_dm_instance)
    

    return tour_seq_fixed, tour_seq_fixed_length

In [7]:
# route similarity

def find_similarity_route(Tour1, Tour2):
    predecessors1 = {Tour1[i]: Tour1[i-1] for i in range(1, len(Tour1))}
    predecessors2 = {Tour2[i]: Tour2[i-1] for i in range(1, len(Tour2))}
    num_changed_predecessors = 0

    for node, pred2 in predecessors2.items():
        pred1 = predecessors1[node]
        if pred1 != pred2:
            num_changed_predecessors += 1
    percent_change = num_changed_predecessors / (len(Tour1)-1) * 100
    sim_pct = 100 - percent_change

    return num_changed_predecessors, sim_pct

In [8]:
# heuristic google OR algo

class TSP:
    def __init__(self, distance_matrix):
        #read distance matrix --> list
        self.dist_mat = distance_matrix
        self.dist_mat = self.round_up(self.dist_mat)
        # Create data model
        self.data = self.create_data_model()
        # Create routing index manager
        self.manager = pywrapcp.RoutingIndexManager(len(self.data['distance_matrix']),
                                            self.data['num_vehicles'], self.data['depot'])
        # Create Routing Model
        self.routing = pywrapcp.RoutingModel(self.manager)
        # Define cost of each arc
        self.transit_callback_index = self.routing.RegisterTransitCallback(self.distance_callback)
        self.routing.SetArcCostEvaluatorOfAllVehicles(self.transit_callback_index)
    
    def round_up(self , lst):
        '''
        input : list of lists having non integral values
        output : list of lists having integral values.
        Function to round up numbers and return as integers
        '''
        rounded_lst = []
        for inner_lst in lst:
            rounded_inner_lst = []
            for num in inner_lst:
                rounded_num = int(round(num))
                rounded_inner_lst.append(rounded_num)
            rounded_lst.append(rounded_inner_lst)
        return rounded_lst        


    def create_data_model(self):
        # Stores the data for the problem
        data = {}
        data['distance_matrix'] = self.dist_mat 
        data['num_vehicles'] = 1
        data['depot'] = 0
        return data

    def distance_callback(self, from_index, to_index):
        # Returns the distance between the two nodes
        from_node = self.manager.IndexToNode(from_index)
        to_node = self.manager.IndexToNode(to_index)
        return self.data['distance_matrix'][from_node][to_node]

    def solve(self):
        # Setting first solution heuristic
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
        # Solve the problem
        solution = self.routing.SolveWithParameters(search_parameters)
        if solution:
            # Get the route as an array
            route = []
            index = self.routing.Start(0)
            while not self.routing.IsEnd(index):
                node = self.manager.IndexToNode(index)
                route.append(node)
                index = solution.Value(self.routing.NextVar(index))
            # Get the objective value
            obj_value = solution.ObjectiveValue()
            # route = self.map_nodes(route)
            return route, obj_value

In [9]:
def map_actual_route(route, mapping):
  tour_mapped = [mapping[a] for a in route]
  return tour_mapped

#Read necessary files

In [11]:
# upload files
all_original = pickle.load(open("/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Coding/Results/all_original_complete.p", "rb"))
warmsen_instances = pickle.load(open("/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Coding/Results/warmsen_instances.p", "rb"))
all_default_tours = pickle.load(open("/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Coding/Results/all_default_tours.p", "rb"))

In [12]:
all_original['Warmsen']['31606-14'].keys()

dict_keys(['start_end_points', 'dm', 'map'])

#Hardware Info

In [13]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmio_stale_data retbleed
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
addres

#Google OR : Tours, Scenarios, Comparison

## Default and Optimized

In [14]:
# should we multiply by 100 then round?

In [15]:
# select sample default tour

default_tour = all_default_tours['Warmsen']['31600-01']

In [16]:
warmsen_instances['Monday']['31600-01']['package_medium'].keys()

dict_keys(['dm', 'map', 'start_end_points', 'volume'])

In [17]:
# select instance to compare

instance_selected_package = warmsen_instances['Monday']['31600-01']['package_medium']
instance_selected_letter = warmsen_instances['Monday']['31600-01']['letter_medium']

In [18]:
print(len(warmsen_instances['Monday']['31600-01']['letter_medium']['map']))
print(len(warmsen_instances['Monday']['31600-01']['letter_high']['map']))
print(len(warmsen_instances['Monday']['31600-01']['letter_low']['map']))

52
64
32


In [19]:
default_tour.keys()

dict_keys(['default_route', 'mapping', 'route_cost'])

In [20]:
%%time

# complete tour improved by solver

# ori_file = '/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Coding/Warmsen_original/dm_31600-01.p'

tsp = TSP(all_original['Warmsen']['31600-01']['dm'])

# tsp = TSP(ori_file)

complete_tour_solver = {}

complete_tour_solver['tour'], complete_tour_solver['cost'] = tsp.solve()

complete_tour_solver['tour'] = map_actual_route(complete_tour_solver['tour'], all_original['Warmsen']['31600-01']['map'])

CPU times: user 23.6 s, sys: 2.17 ms, total: 23.6 s
Wall time: 23.9 s


In [21]:
len(complete_tour_solver['tour'])

427

In [25]:
# 
complete_tour_solver['cost']

25071

In [26]:
25071/3600

6.964166666666666

In [28]:
max(complete_tour_solver['tour'])

750

In [29]:
# pickle.dump(complete_tour_solver, 'warmsen_31600-01_opt.p'), 'wb'))
pickle.dump(complete_tour_solver, open('warmsen_31600-01_opt.p', 'wb'))

In [30]:
complete_tour_solver['tour'][0:5]

[0, 698, 699, 700, 701]

In [31]:
complete_tour_solver['tour'][-5:]

[501, 502, 498, 507, 538]

In [32]:
all_original['Warmsen']['31600-01']['start_end_points']

[0, 0]

In [33]:
# ensure tour has right start and end points
# complete_tour_solver_rev = {}

complete_tour_solver['tour'] = arrange_tour(solver_tour = complete_tour_solver['tour'],
                                            start_end_points = all_original['Warmsen']['31600-01']['start_end_points'])

In [34]:
# after
print(complete_tour_solver['tour'][0:5])
print(complete_tour_solver['tour'][-5:])

[0, 698, 699, 700, 701]
[502, 498, 507, 538, 0]


##Letter Instance

In [35]:
%%time

# skipped from default tour

letter_skip_from_default = {}

letter_skip_from_default['route'], letter_skip_from_default['cost'] = new_tour_sequence_fixed(complete_tour = all_default_tours['Warmsen']['31600-01']['default_route'],
                                            dm_instance = warmsen_instances['Monday']['31600-01']['letter_medium']['dm'],
                                            mapping_pp_needed = warmsen_instances['Monday']['31600-01']['letter_medium']['map'])

CPU times: user 9.62 ms, sys: 914 µs, total: 10.5 ms
Wall time: 22.6 ms


In [36]:
%%time

# skipped from solver complete

letter_skip_from_solver = {}

letter_skip_from_solver['route'], letter_skip_from_solver['cost'] = new_tour_sequence_fixed(complete_tour = complete_tour_solver['tour'],
                                                                                     dm_instance = warmsen_instances['Monday']['31600-01']['letter_medium']['dm'],
                                                                                     mapping_pp_needed = warmsen_instances['Monday']['31600-01']['letter_medium']['map'])

CPU times: user 9.34 ms, sys: 0 ns, total: 9.34 ms
Wall time: 15.2 ms


In [37]:
%%time
# optimize letter instance

tsp = TSP(warmsen_instances['Monday']['31600-01']['letter_medium']['dm'])


letter_tour_solver = {}

letter_tour_solver['tour'], letter_tour_solver['cost'] = tsp.solve()

letter_tour_solver['tour'] = map_actual_route(letter_tour_solver['tour'],
                                              warmsen_instances['Monday']['31600-01']['letter_medium']['map'])

letter_tour_solver['tour'] = arrange_tour(solver_tour = letter_tour_solver['tour'],
                                            start_end_points = all_original['Warmsen']['31600-01']['start_end_points'])

CPU times: user 59.3 ms, sys: 1.66 ms, total: 61 ms
Wall time: 67.3 ms


In [38]:
# similarities
# complete

# optimized complete tour vs default tour
num_changed_predecessors_complete, percent_change_complete = find_similarity_route(all_default_tours['Warmsen']['31600-01']['default_route'],
                                                                                   complete_tour_solver['tour'])
print(num_changed_predecessors_complete, percent_change_complete)

print(complete_tour_solver['cost'])
print(all_default_tours['Warmsen']['31600-01']['route_cost'])

print(all_default_tours['Warmsen']['31600-01']['route_cost'] - complete_tour_solver['cost'])
print(100*(all_default_tours['Warmsen']['31600-01']['route_cost'] - complete_tour_solver['cost'])/complete_tour_solver['cost'])
print(len(all_default_tours['Warmsen']['31600-01']['default_route']), len(complete_tour_solver['tour']))

312 26.932084309133486
25071
26290.29
1219.2900000000009
4.863348091420369
428 428


In [39]:
# similarities
# instance

l_1 = letter_skip_from_solver['route']
l_2 = letter_skip_from_default['route']

# skipped from complete tour vs skipped from default tour
num_changed_predecessors_complete, percent_change_complete = find_similarity_route(l_1,
                                                                                   l_2)
print(num_changed_predecessors_complete, percent_change_complete)

print(letter_skip_from_solver['cost'])
print(letter_skip_from_default['cost'])

print(letter_skip_from_default['cost'] - letter_skip_from_solver['cost'])
print(100*(letter_skip_from_default['cost'] - letter_skip_from_solver['cost'])/letter_skip_from_solver['cost'])
print(len(letter_skip_from_solver['route']), len(letter_skip_from_default['route']))

37 28.84615384615384
7912.6
7063.13
-849.4700000000003
-10.735662108535756
53 53


In [40]:
# similarities
# instance

l_1 = letter_tour_solver['tour']
l_2 = letter_skip_from_default['route']

# skipped from default complete tour vs optimized instance
num_changed_predecessors_complete, percent_change_complete = find_similarity_route(l_1,
                                                                                   l_2)
print(num_changed_predecessors_complete, percent_change_complete)

print(letter_tour_solver['cost'])
print(letter_skip_from_default['cost'])

print(letter_skip_from_default['cost'] - letter_tour_solver['cost'])
print(100*(letter_skip_from_default['cost'] - letter_tour_solver['cost'])/letter_tour_solver['cost'])
print(len(letter_tour_solver['tour']), len(letter_skip_from_default['route']))

38 26.923076923076934
6747
7063.13
316.1300000000001
4.68548984733956
53 53


In [41]:
# similarities
# instance

l_1 = letter_tour_solver['tour']
l_2 = letter_skip_from_solver['route']

# skipped from complete optimized tour vs optimized instance
num_changed_predecessors_complete, percent_change_complete = find_similarity_route(l_1,
                                                                                   l_2)
print(num_changed_predecessors_complete, percent_change_complete)

print(letter_tour_solver['cost'])
print(letter_skip_from_solver['cost'])

print(letter_skip_from_solver['cost'] - letter_tour_solver['cost'])
print(100*(letter_skip_from_solver['cost'] - letter_tour_solver['cost'])/letter_tour_solver['cost'])
print(len(letter_tour_solver['tour']), len(letter_skip_from_solver['route']))

38 26.923076923076934
6747
7912.6
1165.6000000000004
17.275826293167338
53 53


##Package Instance

In [42]:
%%time

# skipped from default tour

package_skip_from_default = {}

package_skip_from_default['route'], package_skip_from_default['cost'] = new_tour_sequence_fixed(complete_tour = all_default_tours['Warmsen']['31600-01']['default_route'],
                                            dm_instance = warmsen_instances['Monday']['31600-01']['package_medium']['dm'],
                                            mapping_pp_needed = warmsen_instances['Monday']['31600-01']['package_medium']['map'])

CPU times: user 12.2 ms, sys: 0 ns, total: 12.2 ms
Wall time: 16.7 ms


In [43]:
%%time

# skipped from solver complete

package_skip_from_solver = {}

package_skip_from_solver['route'], package_skip_from_solver['cost'] = new_tour_sequence_fixed(complete_tour = complete_tour_solver['tour'],
                                                                                     dm_instance = warmsen_instances['Monday']['31600-01']['package_medium']['dm'],
                                                                                     mapping_pp_needed = warmsen_instances['Monday']['31600-01']['package_medium']['map'])

CPU times: user 5.94 ms, sys: 974 µs, total: 6.92 ms
Wall time: 7.07 ms


In [44]:
%%time
# optimize package instance

tsp = TSP(warmsen_instances['Monday']['31600-01']['package_medium']['dm'])


package_tour_solver = {}

package_tour_solver['tour'], package_tour_solver['cost'] = tsp.solve()

package_tour_solver['tour'] = map_actual_route(package_tour_solver['tour'],
                                              warmsen_instances['Monday']['31600-01']['package_medium']['map'])

package_tour_solver['tour'] = arrange_tour(solver_tour = package_tour_solver['tour'],
                                            start_end_points = all_original['Warmsen']['31600-01']['start_end_points'])

CPU times: user 19.3 ms, sys: 3.95 ms, total: 23.3 ms
Wall time: 25.1 ms


In [45]:
# similarities
# instance

p_1 = package_skip_from_solver['route']
p_2 = package_skip_from_default['route']

# skipped from complete tour vs skipped from default tour
num_changed_predecessors_complete, percent_change_complete = find_similarity_route(p_1,
                                                                                   p_2)
print(num_changed_predecessors_complete, percent_change_complete)

print(package_skip_from_solver['cost'])
print(package_skip_from_default['cost'])

print(package_skip_from_default['cost'] - package_skip_from_solver['cost'])
print(100*(package_skip_from_default['cost'] - package_skip_from_solver['cost'])/package_skip_from_solver['cost'])
print(len(package_skip_from_solver['route']), len(package_skip_from_default['route']))

18 28.0
5552.82
4863.22
-689.5999999999995
-12.41891507378232
26 26


In [46]:
# similarities
# instance

p_1 = package_tour_solver['tour']
p_2 = package_skip_from_default['route']

# skipped from default complete tour vs optimized instance
num_changed_predecessors_complete, percent_change_complete = find_similarity_route(p_1,
                                                                                   p_2)
print(num_changed_predecessors_complete, percent_change_complete)

print(package_tour_solver['cost'])
print(package_skip_from_default['cost'])

print(package_skip_from_default['cost'] - package_tour_solver['cost'])
print(100*(package_skip_from_default['cost'] - package_tour_solver['cost'])/package_tour_solver['cost'])
print(len(package_tour_solver['tour']), len(package_skip_from_default['route']))

8 68.0
4688
4863.22
175.22000000000025
3.7376279863481283
26 26


In [47]:
# similarities
# instance

p_1 = package_tour_solver['tour']
p_2 = package_skip_from_solver['route']

# skipped from complete optimized tour vs optimized instance
num_changed_predecessors_complete, percent_change_complete = find_similarity_route(p_1,
                                                                                   p_2)
print(num_changed_predecessors_complete, percent_change_complete)

print(package_tour_solver['cost'])
print(package_skip_from_solver['cost'])

print(package_skip_from_solver['cost'] - package_tour_solver['cost'])
print(100*(package_skip_from_solver['cost'] - package_tour_solver['cost'])/package_tour_solver['cost'])
print(len(package_tour_solver['tour']), len(package_skip_from_solver['route']))

21 16.0
4688
5552.82
864.8199999999997
18.447525597269617
26 26
